In [1]:
import os
import requests
from dotenv import load_dotenv
from typing import Any, Type, List
from langchain_core.tools import BaseTool
from langchain_core.pydantic_v1 import BaseModel, Field
from datetime import datetime

load_dotenv()
CLICKUP_TOKEN = os.getenv("CLICKUP_TOKEN")

In [2]:
headers = {"Authorization": f"{CLICKUP_TOKEN}"}
headers

{'Authorization': 'pk_84141406_KOPJTNTK0QHWINT2HRW0XDC9A3ZC4V6T'}

## Delete Task Tool

In [3]:
from composio.tools.local.clickup.actions.delete_task import DeleteTask, DeleteTaskRequest, DeleteTaskResponse

class DeleteTaskTool(BaseTool):
    name: str = "delete_task_tool"
    description: str = """
    Ferramenta para deletar uma tarefa no ClickUp com base nos parâmetros fornecidos.
    - Deletar Tarefa:
        Invocar: "DeleteTaskTool" com os parâmetros apropriados.
    """
    args_schema: Type[BaseModel] = DeleteTaskRequest
    headers: dict = {"Authorization": f"{CLICKUP_TOKEN}"}

    def __init__(self, **data):
        super().__init__(**data)

    def _run(self, task_id: str, **delete_params) -> Any:
        """Executa a deleção de tarefa no ClickUp"""

        action = DeleteTask()


        url = f"{action.url}{action.path}".format(task_id=task_id)
        params = {key: value for key, value in delete_params.items() if value is not None}

        response = requests.delete(url, headers=self.headers, params=params)

        if response.status_code == 204:
            response_json = {"message": "Task deleted successfully"}
        else:
            try:
                response_json = response.json()
            except requests.JSONDecodeError:
                response_json = {"error": "Invalid JSON response"}
        
        print(f"Response Body: {response_json}")
        return DeleteTaskResponse(data=response_json)

/Users/morossini/anaconda3/envs/secretary/lib/python3.11/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/Users/morossini/anaconda3/envs/secretary/lib/python3.11/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


## Update Task Tool

In [4]:
from composio.tools.local.clickup.actions.update_task import UpdateTask, UpdateTaskRequest, UpdateTaskResponse

class UpdateTaskRequestCustom(UpdateTaskRequest):
    assignees_add: List[int] = Field(
        default=None,
        alias="assignees__add",
        description="",
    )
    assignees_rem: List[int] = Field(
        default=None,
        alias="assignees__rem",
        description="",
    )


class UpdateTaskTool(BaseTool):
    name: str = "update_task_tool"
    description: str = """
    Ferramenta para atualizar uma tarefa no ClickUp com base nos parâmetros fornecidos.
    - Atualizar Tarefa:
        Invocar: "UpdateTaskTool" com os parâmetros apropriados.
    """
    args_schema: Type[BaseModel] = UpdateTaskRequestCustom
    headers: dict = {"Authorization": f"{CLICKUP_TOKEN}"}

    def __init__(self, **data):
        super().__init__(**data)

    def _run(self, task_id: str, **update_params) -> Any:
        """Executa a atualização de tarefa no ClickUp"""

        action = UpdateTask()

        url = f"{action.url}{action.path}".format(task_id=task_id)
        params = {key: value for key, value in update_params.items() if value is not None}

        response = requests.put(url, headers=self.headers, json=params)

        if response.status_code == 200:
            response_json = response.json()
        else:
            try:
                response_json = response.json()
            except requests.JSONDecodeError:
                response_json = {"error": "Invalid JSON response"}
        
        print(f"Response Body: {response_json}")
        return UpdateTaskResponse(data=response_json)

## Agente

In [5]:
from langchain import hub
from langchain_openai import AzureChatOpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent

load_dotenv()

llm = AzureChatOpenAI(model="gpt-4o")
prompt = hub.pull("hwchase17/openai-tools-agent")

In [6]:
prompt_template = f""""\n
You are Jarbas, Project Manager at Cogmo. You are responsible for managing projects and tasks in ClickUp.

Using Tools:
You have access to a variety of tools to help you in your conversations.
Make sure to always pass the correct arguments to each tool.

Project Management:
Use ClickUp to manage projects and tasks. To do that, you need to understand its structure.
Users will refer to lists by name (not always the exact name), so you need to search for the closest match and get its ID.
-> Workspace: The highest level of organization in ClickUp. It contains all of your Spaces.
--> Space: A collection of Folders and Lists. It's a way to group related work together.
---> Folder: Used to group Lists together.
----> List: Used to group tasks together.
-----> Task: The basic unit of work in ClickUp. They can be assigned to people, have due dates, and more.
Important ClickUp IDs (Always use these IDs when interacting with ClickUp):
- Your ClickUp user details:
 - id: 84141406
 - username: Ana Beatriz
 - email: ti@cogmo.com.br
- Cogmo Workspace (aka "team_id"): {os.getenv('CLICKUP_TEAM_ID', 12927880)}
- 'Projetos' Space id {os.getenv('CLICKUP_PROJETOS_SPACE_ID', 54804921)}
- "Agentes Folder id: 90131663060
- "clickup_agent" List id: 901304909413
"""

In [7]:
prompt.messages[0].prompt.template = prompt_template

In [8]:
from langchain.memory import ConversationBufferMemory

tools = [
    UpdateTaskTool(),
    DeleteTaskTool(),
]

agent_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

/Users/morossini/anaconda3/envs/secretary/lib/python3.11/site-packages/pydantic/json_schema.py:2179: PydanticJsonSchemaWarning: Default value alias='assignees__add' alias_priority=2 description='' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/Users/morossini/anaconda3/envs/secretary/lib/python3.11/site-packages/pydantic/json_schema.py:2179: PydanticJsonSchemaWarning: Default value alias='assignees__rem' alias_priority=2 description='' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


# Testes

In [9]:
# load_dotenv()

# agent_executor.invoke(
#     {
#         "input": f"Delete a tarefa 86a4q652d"
#     }
# )

In [10]:
# agent_executor.invoke(
#     {
#         "input": f"Mude o a data da tarefa 86a4q63vx para 2024-09-30"
#     }
# )

In [11]:
# agent_executor.invoke(
#     {
#         "input": f"Quais são os principais ids do ClickUp a que você tem acesso?"
#     }
# )

In [12]:
# Envie uma requisição GET ao endpoint /user
response = requests.get('https://api.clickup.com/api/v2/user', headers=headers)

In [13]:
response.content

b'{"user":{"id":84141406,"username":"Ana Beatriz","email":"ti@cogmo.com.br","color":"#455963","profilePicture":"https://attachments.clickup.com/profilePictures/84141406_ZkZ.jpg","initials":"AB","week_start_day":1,"global_font_support":true,"timezone":"America/Sao_Paulo"}}'